In [7]:
import numpy as np
import arviz as az
import matplotlib
import matplotlib.pyplot as plt

from glob import glob
from tqdm.notebook import tqdm
import os
import json
import pandas as pd
from cmdstanpy import cmdstan_path, CmdStanModel
import cv2
import json

In [ ]:
def fit_model(data_path,outpath,path):
    print(os.path.join(path,'models/hier_s_warped.stan'))
    model = CmdStanModel(stan_file=os.path.join(path,'models/hier_s_warped.stan'),
                        stanc_options={'allow_undefined':True},
                        user_header=os.path.join(path,'models/add_iter.hpp'))
    fit = model.sample(data=data_path,parallel_chains=4,chains=4,
                    show_progress='notebook',adapt_delta=0.99,max_treedepth=15)
    print(fit.diagnose())

    draws = fit.draws_pd()

    out = os.path.join(path,outpath)
    if not os.path.exists(out):
        os.mkdir(out)

    draws.to_csv(os.path.join(out,'fit.csv'))

## MCMC sampling

Running all takes hours.
8 hours on AMD Ryzen 7 5800X.

In [ ]:
full_path = '/home/lab/projects/spatial'
paths = glob(os.path.join(full_path,'data/spatial_data/raw/*.json'))

out = 'results/fits'
for idx,path in enumerate(paths):
    data_path = os.path.split(path)[1]
    outpath = os.path.join(out,'fit_{}'.format(data_path.split('.')[0]))
    print(data_path,outpath)
    fit_model(path,outpath,full_path)

### Filter relevant parameters for the visualizations

In [ ]:
# caf, control, repeat, incubation
pairs = [['231104_CAF','231104_Contr',1,1],
        ['231105_CAF','231105_Contr',1,2],
        ['231106_CAF','231106_Contr',1,3],
        ['240723_CAF','240723_Contr',2,1],
        ['240724_CAF','240724_Contr',2,2],
        ['240725_CAF','240725_Contr',2,3],
        ['240813_CAF','240813_Contr',3,1],
        ['240814_CAF','240814_Contr',3,2],
        ['240815_CAF','240815_Contr',3,3],
        ]

In [ ]:
fits = []
fgs = []
fphis = []
for i,j,repeat,incu in pairs:
    date = i.split('_')[0]
    print(i,j)
    fit_treat = pd.read_csv(os.path.join(full_path,'results','fits',f'fit_{i}','fit.csv'))
    fit_treat['condition'] = 'CAF'
    fit_contr = pd.read_csv(os.path.join(full_path,'results','fits',f'fit_{j}','fit.csv'))
    fit_contr['condition'] = 'control'

    tmp = fit_treat.filter(regex='F_g').mean(axis=0).to_frame()
    tmp['condition'] = 'CAF'
    tmp['date'] = date
    tmp['incubation'] = incu
    tmp['repetition'] = repeat
    tmp2 = fit_contr.filter(regex='F_g').mean(axis=0).to_frame()
    tmp2['condition'] = 'control'
    tmp2['date'] = date
    tmp2['incubation'] = incu
    tmp2['repetition'] = repeat

    tmp3 = fit_treat.filter(regex='F_phi').mean(axis=0).to_frame()
    tmp3['condition'] = 'CAF'
    tmp3['date'] = date
    tmp3['incubation'] = incu
    tmp3['repetition'] = repeat
    tmp4 = fit_contr.filter(regex='F_phi').mean(axis=0).to_frame()
    tmp4['condition'] = 'control'
    tmp4['date'] = date
    tmp4['incubation'] = incu
    tmp4['repetition'] = repeat
    fgs.append(pd.concat([tmp,tmp2]))
    fphis.append(pd.concat([tmp3,tmp4]))
    fit = pd.concat([fit_treat.filter(regex='alpha_g$|alpha_g_sigma$|alpha_phi$|alpha_phi_sigma$|offset_g$|offset_phi$|offset_g_std$|offset_phi_std$|condition'),
                fit_contr.filter(regex='alpha_g$|alpha_g_sigma$|alpha_phi$|alpha_phi_sigma$|offset_g$|offset_phi$|offset_g_std$|offset_phi_std$|condition')])
    fit['date'] = date
    fit['repetition'] = repeat
    fit['incubation'] = incu
    fits.append(fit)

fits = pd.concat(fits)
fgs = pd.concat(fgs)
fphis = pd.concat(fphis)

softplus = lambda x: np.log1p(np.exp(x))
sigmoid = lambda x: 1/(1 + np.exp(-x)) 

fits['alpha_g_s'] = softplus(fits['alpha_g'])
fits['alpha_phi_s'] = softplus(fits['alpha_phi'])
fits['offset_g_s'] = softplus(fits['offset_g'])
fits['offset_g'] = fits['offset_g']
fits['offset_phi_s'] = np.arcsin(sigmoid(fits['offset_phi']))

These samples can be used to generate all results in the paper

In [ ]:
fits.to_csv(os.path.join(full_path,'data','processed','fits_2.csv'))
fgs.to_csv(os.path.join(full_path,'data','processed','fgs_2.csv'))
fphis.to_csv(os.path.join(full_path,'data','processed','fphis_2.csv'))